<a href="https://colab.research.google.com/github/Herry-Suen/DecryptLogin/blob/master/Data_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#!/user/bin/python
# -*- coding: UTF-8 -*-
'''
Student Name: ZHU SHASHA
填写你们组员的名字

'''
%pip install mpl_finance


import pandas as pd
import numpy as np
import pandas_datareader as web
import os
import datetime
import time
import matplotlib.pyplot as plt
from matplotlib import ticker as mticker
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)









ERROR: Could not find a version that satisfies the requirement pylab (from versions: none)
ERROR: No matching distribution found for pylab
  Created wheel for pytalib: filename=pytalib-0.0.2-cp36-none-any.whl size=16928 sha256=80587484fbfaed666c0c46fea2fa37e1fb0cc16313126618cecf76a671b1efae
  Stored in directory: /root/.cache/pip/wheels/93/17/20/6bedecea8148d420ed01630dc73c40b9f6e22bd80e17b20499
Successfully built pytalib
  ERROR: Failed building wheel for talib
  Running setup.py clean for talib
Failed to build talib
    Running setup.py install for talib ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-xdsx5f_r/talib/setup.py'"'"'; __file__='"'"'/tmp/pip-install-xdsx5f_r/talib/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-q5

In [6]:

def get_today():
    today = time.localtime(time.time())
    today_year  = today.tm_year
    today_month = today.tm_mon
    today_date = today.tm_mday
    today_format = datetime.datetime(today_year, today_month, today_date)
    today_format = today_format.strftime('%Y%m%d') #20191001
    return today_format

class GetData(object):
    def __init__(self,Name,Startdate,Enddate,LabelName,Datasource='yahoo'):
        self.Name = Name
        self.Startdate = Startdate
        self.Enddate = Enddate
        self.Datasource  = Datasource
        self.LabelName = LabelName
        New_Startdate_format = time.strptime(self.Startdate,'%Y%m%d')
        New_Enddate_format = time.strptime(self.Enddate,'%Y%m%d')
        New_Startdate = datetime.datetime(New_Startdate_format.tm_year,New_Startdate_format.tm_mon,New_Startdate_format.tm_mday).strftime('%m/%d/%Y')
        New_Enddate = datetime.datetime(New_Enddate_format.tm_year,New_Enddate_format.tm_mon,New_Enddate_format.tm_mday).strftime('%m/%d/%Y')
        self.NewStartdate = New_Startdate
        self.NewEnddate = New_Enddate
    def DownloadData(self):
        DataInformation = web.DataReader(self.Name,self.Datasource,self.NewStartdate,self.NewEnddate)
        Trade_Date_List = []
        for TradeDate in range(len(DataInformation.index)):
            Trade_Date_Format = time.strptime(str(DataInformation.index[TradeDate]), '%Y-%m-%d %H:%M:%S')
            Trade_Date = datetime.datetime(Trade_Date_Format.tm_year, Trade_Date_Format.tm_mon,
                                           Trade_Date_Format.tm_mday).strftime('%Y%m%d')
            Trade_Date_List.append(Trade_Date)
        Index_Trade_Date_List = []
        for TradeDate in range(len(DataInformation.index)):
            Index_Trade_Date_Format = time.strptime(str(DataInformation.index[TradeDate]), '%Y-%m-%d %H:%M:%S')
            Index_Trade_Date = datetime.datetime(Index_Trade_Date_Format.tm_year, Index_Trade_Date_Format.tm_mon,
                                           Index_Trade_Date_Format.tm_mday).strftime('%Y-%m-%d')
            Index_Trade_Date_List.append(Index_Trade_Date)
        DataInformation['IndexTradeDate'] = Index_Trade_Date_List
        DataInformation['TradeDate'] = Trade_Date_List
        DataInformation['Name'] = self.LabelName
        all_columns = DataInformation.columns.tolist()

        all_columns.pop(-1)
        all_columns.insert(0, 'Name')
        all_columns.pop(-1)
        all_columns.insert(1, 'TradeDate')
        all_columns.pop(-1)
        all_columns.insert(2, 'IndexTradeDate')

        DataInformation = DataInformation.reindex(columns = all_columns)
        CurrentFilePath = GetData.SaveData(self,DataInformation)
        print('Data has already been downloaded from %s'%self.Datasource , '!')
        print('Information : \n','Name : %s'%self.Name,'\n','Columns : ',all_columns,'\n','Total index ',len(DataInformation.index),'\n'
              ,'Saved Path in csv: ',CurrentFilePath[0],'\n','Saved Path in excel:',CurrentFilePath[1])
        return DataInformation
    def GetCurrentPath(self):
        CurrentFilePath = os.getcwd()
        return CurrentFilePath
    def SaveData(self,DataInformation):
        CurrentFilePath = GetData.GetCurrentPath(self)
        CurrentFilePath = CurrentFilePath + str(r'\DataBase')
        if not os.path.exists(CurrentFilePath):
            os.makedirs(CurrentFilePath)
        CurrentFilePath_csv = CurrentFilePath + str(r'\Csv')
        if not os.path.exists(CurrentFilePath_csv):
            os.makedirs(CurrentFilePath_csv)
        CurrentFilePath_excel = CurrentFilePath + str(r'\Xlsx')
        if not os.path.exists(CurrentFilePath_excel):
            os.makedirs(CurrentFilePath_excel)
        CurrentFilePath_csv = CurrentFilePath_csv + str(r'\\') + str(self.LabelName)
        CurrentFilePath_csv = CurrentFilePath_csv + str(r'.csv')
        CurrentFilePath_excel = CurrentFilePath_excel + str(r'\\')+ str(self.LabelName)
        CurrentFilePath_excel = CurrentFilePath_excel + str(r'.xlsx')
        DataInformation.to_csv(CurrentFilePath_csv,encoding='utf_8_sig')
        DataInformation.to_excel(excel_writer=CurrentFilePath_excel, encoding='utf_8_sig')
        return CurrentFilePath_csv,CurrentFilePath_excel
    def ReadDataFromCsv(self,path):
        DataInformation = pd.read_csv(path)
        return DataInformation
    def ReadDataFromExcel(self,path):
        DataInformation = pd.read_excel(path)
        return  DataInformation
    def __str__(self):
        return 'Name : '+str(self.LabelName)+'\n'+'Start Date : '+str(self.NewStartdate)+'\n'+'End Date : '+str(self.NewEnddate)+'\n'+'Data Source : '+str(self.Datasource)



In [8]:


enddate = get_today() # or you can enddate = '20201212'
startdate = '20140101'


Portfolio_data = ['AAPL','FB','BABA','NFLX','MSFT','SPLK','SE','BILI','JD','FIVN']
for data in Portfolio_data:
  Portfolio = GetData(data,startdate,enddate,data,'yahoo')
  DataInformation = Portfolio.DownloadData()


SP = GetData('^GSPC',startdate, enddate, 'S_P500')
DataInformation_SP = SP.DownloadData()


Data has already been downloaded from yahoo !
Information : 
 Name : AAPL 
 Columns :  ['Name', 'TradeDate', 'IndexTradeDate', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'] 
 Total index  1752 
 Saved Path in csv:  /content\DataBase\Csv\\AAPL.csv 
 Saved Path in excel: /content\DataBase\Xlsx\\AAPL.xlsx
Data has already been downloaded from yahoo !
Information : 
 Name : FB 
 Columns :  ['Name', 'TradeDate', 'IndexTradeDate', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'] 
 Total index  1752 
 Saved Path in csv:  /content\DataBase\Csv\\FB.csv 
 Saved Path in excel: /content\DataBase\Xlsx\\FB.xlsx
Data has already been downloaded from yahoo !
Information : 
 Name : BABA 
 Columns :  ['Name', 'TradeDate', 'IndexTradeDate', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'] 
 Total index  1572 
 Saved Path in csv:  /content\DataBase\Csv\\BABA.csv 
 Saved Path in excel: /content\DataBase\Xlsx\\BABA.xlsx
Data has already been downloaded from yahoo !
Information : 
 Name 

# New Section

# New Section